In [ ]:
!pip install asterix_decoder

from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive"

In [ ]:
import asterix
import pandas as pd
from pandas import ExcelWriter
import math

#Importamos el archivo de datos:
sample_filename = "./gdrive/My Drive/TFG/ArchivosAsterix/200223-cen-080001_PAR1_MODO_S.ast"

#Abrimos el archivo de datos y lo leemos:
with open(sample_filename, "rb") as f:
    data = f.read()
    data = data[0:7000000]      #Número de datos que analizaremos
    parsed = asterix.parse(data)

    #Definimos las columnas que tendrá nuestro Excel final en forma de listas: 
    AID = []
    FID = []   
    TOD = []
    X = []
    Y = []
    ALT = []
    MACH = []
    TAS = []
    GS = []
    HDG = []
    TRACK = []
    ROLL = []
    WS = []
    WD = []
    TEMP = []
    PRES = []

    #Vamos a seleccionar solo los mensajes que contienen todos los campos que nos interesan:
    for packet in parsed:
      if packet['crc'] != '00000000':
        if 'I140' in packet  and 'I250' in packet and 'I200' in packet and 'I220' in packet and 'I240' in packet and 'I240' in packet and 'I040' in packet:
          if len(packet['I250']) == 3 and len(packet['I250'][0]) >= 15 and len(packet['I250'][1]) >= 11 and len(packet['I250'][1]) >= 11:
            if 'MACH'in packet['I250'][2] and packet['I250'][2]['MACH']['val'] >= 0.1 and packet['I250'][1]['TAS']['val'] != 0 and packet['I250'][2]['IAS']['val'] != 0:
              
              h = packet['I250'][0]['MCP_ALT']['val']/3.2808399   #de ft. a metros
              vTAS_50 = packet['I250'][1]['TAS']['val']/1.944     #de nt. a m/s
              vIAS = packet['I250'][2]['IAS']['val']/1.944
              vGS = packet['I250'][1]['GS']['val']/1.944
              M = packet['I250'][2]['MACH']['val']
              Rho = packet['I040']['RHO']['val']
              Theta = packet['I040']['THETA']['val']
                         
              AID.append(packet['I220']['ACAddr']['val'])
              FID.append(packet['I240']['TId']['val'])
              TOD.append(packet['I140']['ToD']['val'])
              ALT.append(packet['I250'][0]['MCP_ALT']['val'])
              MACH.append(M)
              GS.append(vGS)
              ROLL.append(packet['I250'][1]['RA']['val'])

              #Calculamos coordenadas x e y que identifican la pos. de la aeronave respecto al radar:
              x = (Rho * math.sin(math.radians(Theta))) * 1852   # Multiplicamos por 1852 para pasar de NM a m
              y = (Rho * math.cos(math.radians(Theta))) * 1852

              X.append(x)
              Y.append(y)

              #Guardamos los valores positivos del Heading y el ángulo de Track:
              if packet['I250'][2]['HDG']['val'] < 0:
                packet['I250'][2]['HDG']['val'] = 360 + packet['I250'][2]['HDG']['val']
                HDG.append(packet['I250'][2]['HDG']['val'])
              else:
                HDG.append(packet['I250'][2]['HDG']['val'])

              if packet['I250'][1]['TTA']['val'] < 0:
                packet['I250'][1]['TTA']['val'] = 360 + packet['I250'][1]['TTA']['val']
                TRACK.append(packet['I250'][1]['TTA']['val'])
              else:
                TRACK.append(packet['I250'][1]['TTA']['val'])


              #Definimos constantes para calcular la velocidad y dirección del viento:
              P_0 = 101325   #Pa
              rho_0 = 1.225  #kg/m**3
              a_0 = 340.29   #m/s
              T_0 = 288.15   #K
              R = 287.05     #J/(kg*K)
              g_0 = 9.81     #m/s**2
              gamma = 1.4

              P = P_0*math.exp(-(g_0*h)/(R*T_0))
              PRES.append(P)
              
              if M < 0.3:
                T = (vTAS_50**2)*P/((vIAS**2)*rho_0*R)
                TEMP.append(T)
              else:
                T = (vTAS_50**2)*T_0/((M**2)*(a_0**2))
                TEMP.append(T)

              rho = P/(R*T)
              if M < 0.3:
                vTAS = vIAS*math.sqrt(rho_0/rho)
                TAS.append(vTAS)
              else:
                vTAS = M*a_0*math.sqrt(T/T_0)
                TAS.append(vTAS)

              vTASx = vTAS * math.sin(math.radians(packet['I250'][2]['HDG']['val']))
              vTASy = vTAS * math.cos(math.radians(packet['I250'][2]['HDG']['val']))
              vGSx = vGS * math.sin(math.radians(packet['I250'][1]['TTA']['val']))
              vGSy = vGS * math.cos(math.radians(packet['I250'][1]['TTA']['val']))
   
              vWx = vGSx - vTASx
              vWy = vGSy - vTASy
              vw = math.sqrt(vWx**2+vWy**2)
              WS.append(vw)
              
              if vWx >= 0 and vWy > 0 and vWy != 0:
                wAngle = math.degrees(math.atan(vWx/vWy))
                WD.append(wAngle)
              elif vWx >= 0 and vWy < 0 and vWy != 0:
                wAngle = 180 - math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWx < 0 and vWy > 0 and vWy != 0:
                wAngle = 360 - math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWx < 0 and vWy < 0 and vWy != 0:
                wAngle = 180 + math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWy == 0 and vWx > 0:
                wAngle = 90
                WD.append(wAngle)
              elif vWy == 0 and vWx < 0:
                wAngle = 180
                WD.append(wAngle)
              elif vWy == 0 and vWx == 0:
                wAngle = 0
                WD.append(wAngle)


    #Creamos un dicionario con las listas:
    datos = {'Aircraft ID':AID,'Flight ID':FID,'Time':TOD,'X [m]':X,'Y [m]':Y,'ALT [ft.]':ALT,'MACH':MACH,'TAS [m/s]':TAS,'GS [m/s]':GS,'HDG':HDG,'TRACK':TRACK,'Wind Speed [m/s]':WS,'Wind direc.':WD,'ROLL':ROLL,'Temperature [K]':TEMP,'Pressure [Pa]':PRES}
    df = pd.DataFrame.from_dict(datos, orient='index')
    df = df.transpose()

    #Creamos el Excel con nuestro dicionario:
    writer = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/TablaModeS_08.xlsx")
    df.to_excel(writer, 'Datos Mode-S', index=False)
    writer.save()

    print('Fin')

Fin


In [ ]:
import asterix
import pandas as pd
from pandas import ExcelWriter
import math

#Importamos el archivo de datos:
sample_filename = "./gdrive/My Drive/TFG/ArchivosAsterix/200223-cen-120001_PAR1_MODO_S.ast"

#Abrimos el archivo de datos y lo leemos:
with open(sample_filename, "rb") as f:
    data = f.read()
    data = data[0:7000000]      #Número de datos que analizaremos
    parsed = asterix.parse(data)

    #Definimos las columnas que tendrá nuestro Excel final en forma de listas: 
    AID = []
    FID = []   
    TOD = []
    X = []
    Y = []
    ALT = []
    MACH = []
    TAS = []
    GS = []
    HDG = []
    TRACK = []
    ROLL = []
    WS = []
    WD = []
    TEMP = []
    PRES = []

    #Vamos a seleccionar solo los mensajes que contienen todos los campos que nos interesan:
    for packet in parsed:
      if packet['crc'] != '00000000':
        if 'I140' in packet  and 'I250' in packet and 'I200' in packet and 'I220' in packet and 'I240' in packet and 'I240' in packet and 'I040' in packet:
          if len(packet['I250']) == 3 and len(packet['I250'][0]) >= 15 and len(packet['I250'][1]) >= 11 and len(packet['I250'][1]) >= 11:
            if 'MACH'in packet['I250'][2] and packet['I250'][2]['MACH']['val'] >= 0.1 and packet['I250'][1]['TAS']['val'] != 0 and packet['I250'][2]['IAS']['val'] != 0:
              
              h = packet['I250'][0]['MCP_ALT']['val']/3.2808399   #de ft. a metros
              vTAS_50 = packet['I250'][1]['TAS']['val']/1.944     #de nt. a m/s
              vIAS = packet['I250'][2]['IAS']['val']/1.944
              vGS = packet['I250'][1]['GS']['val']/1.944
              M = packet['I250'][2]['MACH']['val']
              Rho = packet['I040']['RHO']['val']
              Theta = packet['I040']['THETA']['val']
                         
              AID.append(packet['I220']['ACAddr']['val'])
              FID.append(packet['I240']['TId']['val'])
              TOD.append(packet['I140']['ToD']['val'])
              ALT.append(packet['I250'][0]['MCP_ALT']['val'])
              MACH.append(M)
              GS.append(vGS)
              ROLL.append(packet['I250'][1]['RA']['val'])

              #Calculamos coordenadas x e y que identifican la pos. de la aeronave respecto al radar:
              x = (Rho * math.sin(math.radians(Theta))) * 1852   # Multiplicamos por 1852 para pasar de NM a m
              y = (Rho * math.cos(math.radians(Theta))) * 1852

              X.append(x)
              Y.append(y)

              #Guardamos los valores positivos del Heading y el ángulo de Track:
              if packet['I250'][2]['HDG']['val'] < 0:
                packet['I250'][2]['HDG']['val'] = 360 + packet['I250'][2]['HDG']['val']
                HDG.append(packet['I250'][2]['HDG']['val'])
              else:
                HDG.append(packet['I250'][2]['HDG']['val'])

              if packet['I250'][1]['TTA']['val'] < 0:
                packet['I250'][1]['TTA']['val'] = 360 + packet['I250'][1]['TTA']['val']
                TRACK.append(packet['I250'][1]['TTA']['val'])
              else:
                TRACK.append(packet['I250'][1]['TTA']['val'])


              #Definimos constantes para calcular la velocidad y dirección del viento:
              P_0 = 101325   #Pa
              rho_0 = 1.225  #kg/m**3
              a_0 = 340.29   #m/s
              T_0 = 288.15   #K
              R = 287.05     #J/(kg*K)
              g_0 = 9.81     #m/s**2
              gamma = 1.4

              P = P_0*math.exp(-(g_0*h)/(R*T_0))
              PRES.append(P)
              
              if M < 0.3:
                T = (vTAS_50**2)*P/((vIAS**2)*rho_0*R)
                TEMP.append(T)
              else:
                T = (vTAS_50**2)*T_0/((M**2)*(a_0**2))
                TEMP.append(T)

              rho = P/(R*T)
              if M < 0.3:
                vTAS = vIAS*math.sqrt(rho_0/rho)
                TAS.append(vTAS)
              else:
                vTAS = M*a_0*math.sqrt(T/T_0)
                TAS.append(vTAS)

              vTASx = vTAS * math.sin(math.radians(packet['I250'][2]['HDG']['val']))
              vTASy = vTAS * math.cos(math.radians(packet['I250'][2]['HDG']['val']))
              vGSx = vGS * math.sin(math.radians(packet['I250'][1]['TTA']['val']))
              vGSy = vGS * math.cos(math.radians(packet['I250'][1]['TTA']['val']))
   
              vWx = vGSx - vTASx
              vWy = vGSy - vTASy
              vw = math.sqrt(vWx**2+vWy**2)
              WS.append(vw)
              
              if vWx >= 0 and vWy > 0 and vWy != 0:
                wAngle = math.degrees(math.atan(vWx/vWy))
                WD.append(wAngle)
              elif vWx >= 0 and vWy < 0 and vWy != 0:
                wAngle = 180 - math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWx < 0 and vWy > 0 and vWy != 0:
                wAngle = 360 - math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWx < 0 and vWy < 0 and vWy != 0:
                wAngle = 180 + math.degrees(math.atan(abs(vWx/vWy)))
                WD.append(wAngle)
              elif vWy == 0 and vWx > 0:
                wAngle = 90
                WD.append(wAngle)
              elif vWy == 0 and vWx < 0:
                wAngle = 180
                WD.append(wAngle)
              elif vWy == 0 and vWx == 0:
                wAngle = 0
                WD.append(wAngle)


    #Creamos un dicionario con las listas:
    datos = {'Aircraft ID':AID,'Flight ID':FID,'Time':TOD,'X [m]':X,'Y [m]':Y,'ALT [ft.]':ALT,'MACH':MACH,'TAS [m/s]':TAS,'GS [m/s]':GS,'HDG':HDG,'TRACK':TRACK,'Wind Speed [m/s]':WS,'Wind direc.':WD,'ROLL':ROLL,'Temperature [K]':TEMP,'Pressure [Pa]':PRES}
    df = pd.DataFrame.from_dict(datos, orient='index')
    df = df.transpose()

    #Creamos el Excel con nuestro dicionario:
    writer = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/TablaModeS_12.xlsx")
    df.to_excel(writer, 'Datos Mode-S', index=False)
    writer.save()

    print('Fin')

Fin
